In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
import re
import yaml

In [2]:
db = os.environ.get('PSQL_DATABASE')
host = 'localhost'
user ='user'
port = '5432'
psql_engine = f'postgresql://{user}:@{host}:{port}/{db}'

In [3]:
exlcude_forums = [9, 10, 14, 18]

In [4]:
exlcude_forums_where = ', '.join([str(fid) for fid in exlcude_forums])
with open("get_forum_list.sql", "r") as f:
    sql_template = f.read()
slq_statement = ''.join(sql_template).format(exlcude_forums = exlcude_forums_where)

In [6]:
df = pd.read_sql(slq_statement, psql_engine)

In [10]:
with open('forum_checklist.yml', 'w') as f:
    f.write(
        yaml.dump(
            df.forum_id.to_list()
        )
    )

In [ ]:
def map_smiely_code(value):
    result = value
    match value:
        case ':D':
            result = ':grin:'
        case ':)':
            result = ':smile:'
    return result


In [ ]:
def replace_smilies(txt):
    pattern = r'\<\!\-\- s([^-]+) \-\-\>\<img +src\=\"\{SMILIES\_PATH\}\/[a-zA-Z0-9\.\_]+\"(?: +alt\=\"[^\"]+\")?(?: +title\=\"[^"]+\")? \/\>\<\!\-\- s[^-]+ \-\-\>'
    txt = re.split(
        pattern,
        txt
    )
    txt = [value if index%2 == 0 else  map_smiely_code(value) for index, value in enumerate(txt)]

    return ''.join(txt)

In [ ]:
df['forum_description'] = df['forum_description'].apply(lambda x: replace_smilies(x))

In [ ]:
all_forums = []
groups = df[['grp_id', 'grp_name']].drop_duplicates().T.to_dict().values()

for grp in groups:

    grp_id = grp['grp_id']
    grp_name = grp['grp_name']

    forum_subset = df[df['grp_id'] == grp_id]

    forum_group = grp.copy()
    forum_group['fora'] = [
        {
            k : v
            for k, v in forum.items()
            if k not in ['grp_id', 'grp_name']
        }
        for forum in forum_subset.T.to_dict().values()
    ]
    all_forums.append(forum_group)

In [ ]:
yml_path = os.path.join('..', '_tests', '_data', 'lista_for.yml')
with open(yml_path, 'w') as f:
    f.write(yaml.dump(all_forums))

In [ ]:
md_template = '''---
layout: forum_thread_list
forum: {fid}
permalink: /forum/{fid}
pagination: 
  enabled: true
  collection: threads
  per_page: 25
  sort_field: thread_order 
  category: '{fid}'
  trail: 
    before: 2
    after: 2
breadcrumbs:
    - name: Lista For
      href: /forum_list.html
    - name: {fname}
      href: /forum/{fid}/index.html
---'''

In [ ]:
for i in df.T.to_dict().values():

    fid = i['forum_id']
    fname=i['forum_name']

    md_content = md_template.format(fid=fid, fname=fname)
    md_path = os.path.join('..', '_tests', 'forum', f'forum_{fid}.md')

    with open(md_path, 'w') as f:
        f.write(md_content)